In [1]:
%cd ..
%reload_ext autoreload
%autoreload 2

/Users/saattrupdan/gitsky/dr_hatespeech


In [2]:
import pandas as pd
from transformers import pipeline
from src.data import load_data
import re

pd.set_option("max_colwidth", None)

## Explore the cleaned data

In [3]:
df = load_data()
print(f"Loaded {len(df):,} posts.")
df.head(5)

Loaded 4,221,221 posts.


,account,text,url,date,action,post_id,comment_id,reply_comment_id
0,dr nyheder,Fatma Kücük Öztürk,None,2021-03-06,none,NaN,NaN,NaN
1,dr nyheder,"Bente Jørgensen godt vi ikke lærte det da vi gik i skole, ja at brokke sig",None,2021-03-06,none,NaN,NaN,NaN
2,dr nyheder,"Når man har læst om, hvad Bolsenaro har udtalt om homoseksuelle, kvinder, klima og andre ting, kan intet overraske fra den kant. Han får nærmest Trump til at virke venstreorienteret.",None,2021-03-06,none,NaN,NaN,NaN
3,dr nyheder,Bente Kristoffersen det beklager jeg så. Der har blot været så mange kommentarer der har været forstokkede. Lagde faktisk ikke mærke til grinesmileyen før du lige nævner det nu. Håber du får en god dag,None,2021-03-06,none,NaN,NaN,NaN
4,dr nyheder,Mohammad Askar Og bedetæpper,None,2021-03-06,none,NaN,NaN,NaN


In [20]:
len(df[df.text.str.contains('[EMAIL]', regex=False)])

0

## Explore the weakly supervised labels

In [4]:
ws_df = pd.read_parquet("data/processed/data_2022_weakly_supervised.parquet")
ws_df.head()

,account,text,url,date,action,post_id,comment_id,reply_comment_id,label
0,dr nyheder,Fatma Kücük Öztürk,None,2021-03-06,none,NaN,NaN,NaN,0
1,dr nyheder,"Bente Jørgensen godt vi ikke lærte det da vi gik i skole, ja at brokke sig",None,2021-03-06,none,NaN,NaN,NaN,0
2,dr nyheder,"Når man har læst om, hvad Bolsenaro har udtalt om homoseksuelle, kvinder, klima og andre ting, kan intet overraske fra den kant. Han får nærmest Trump til at virke venstreorienteret.",None,2021-03-06,none,NaN,NaN,NaN,0
3,dr nyheder,Bente Kristoffersen det beklager jeg så. Der har blot været så mange kommentarer der har været forstokkede. Lagde faktisk ikke mærke til grinesmileyen før du lige nævner det nu. Håber du får en god dag,None,2021-03-06,none,NaN,NaN,NaN,0
4,dr nyheder,Mohammad Askar Og bedetæpper,None,2021-03-06,none,NaN,NaN,NaN,0


In [6]:
ws_df.query("label == 1").head(3)

,account,text,url,date,action,post_id,comment_id,reply_comment_id,label
14,dr nyheder,hvis jeg skulle demonstrer skulle det sku være for klima og ikke med de her idioter,None,2021-03-14,none,NaN,NaN,NaN,1
68,dr nyheder,"Niels Christian Baark er det virkelig sådan et samfund du ønsker at leve i og have dine børn til at opvokse i? Sidst jeg kiggede, var vi et frit demokratisk samfund med ytringsfrihed etc. Men lige nu synes jeg vi kommer nærmere og nærmere østtyske tilstande, hvor folk hentes i deres hjem for at poste et billede af en dukke. Påtvunget en mundble som sundhedsmyndighederne selv har sagt har ringe eller ingen værdi, indskrænket personlig frihed, listen fortsætter og fortsætter... Er dødtræt af forsmåede danskere der ikke tør eller vil stille spørgsmål til alle de idiotiske retningslinier og love der udstikkes i forbindelse med denne influenza lignende sygdom, som på over et år har slået 2400 ihjel med en gennemsnitsalder på over 80 år. Hvor er forbud mod rygning det slår 14000 ihjel om året henne af?",None,2021-04-06,none,NaN,NaN,NaN,1
89,dr nyheder,"det var også lovligt, at slå sine børn og sin kone for ikke ret lang tid siden i danmark, danmark er officielt ikke et udviklingsland, men vi(alle lande er vel i en udvikling) vi(bl.a danske service organer incl folketinget) flueknepper og spilder unødvendige ressourcer på komedier for folket og har glemt, at de skal servicere befolkningen(vel også i global forstand) skabe grundlag for, at kunne se udover sin egen næsetip eller bare på lidt længere sigt ... undskyldningen er ofte økonomisk knaphed og manglede overskud(måske er de på en eller anden måde en sikkerhedsrisiko.. ...), jeg er vel også er risiko ...da jeg ønsker kvalitet og medmenneskelighed, fremfor sikkerhed og nyt .. fremmede er folk jeg bare ikke kender endnu, men vil de fleste mennesker ikke bare være en del af og ikke ødelægge det for andre .. .. vi begår alle fejl og må må tage ansvar for disse, men vi er jo også bare mennesker ... måske kan vi overveje om vi vil, at der er plads til, at andre måske også bidrager, selvom de kommer fra en anden landsby .... eller er vi bare alle sammen på vej ud over afgrunden(det føles lidt trygt, at gøre det sammen) som lemminge i et nyt samtalebadeværelse ;-) også jeg sidder fast med hovedet oppe i røven mens jeg fejler, så jeg skal vel også op til prøven om retten til dansk statborgerskab ... overvejer indimellem, at melde mig ud af danmark og står ikke op for disse såkaldte danske værdier sat op regneark - har vi råd til, at ikke kunne i fællesskab?",None,2021-04-22,none,NaN,NaN,NaN,1
